In [ ]:
# if running on google colab, uncomment and run these lines if you want to mount your google drive
# so you can access your drive files from within the notebook, and save written files to drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python --version

import tensorflow
print("tensorflow version: " + tensorflow.__version__)
import tensorflow.keras
print("keras version: " + tensorflow.keras.__version__)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input, Activation, add
from tensorflow.keras import regularizers

from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import cv2 as cv
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import glob
import time
from sklearn.model_selection import train_test_split

In [4]:
# define PilotNet architecture

# pilotnet parameters for network dimension:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 240, 200, 3       # If you decide to change your cropping you'll need to change these!
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

# magnitude of your estimated steering angle correction for the left/right cameras
OFFSET_STEERING_ANGLE = 0.1                                   # NOTE: TUNE THIS and use two values if the left/right camera angles are not symmetrical

# some convenience functions for preprocessing frames:

def crop(image):
    """
    assumes 320x240 input (images are 320 (width) x 240 (height) from the RGB cam), 
    resizes to 200x240
    """
    #      (original - target)
    # columns: (320 - 200) /2 == 60 
    return image[:, 60:-60] 

def pilotnet_crop(image):
    """
    assumes 320x240 input (images are 320 (width) x 240 (height) from the RGB cam), 
    resizes to 200x66 (original Nvidia paper uses these dimensions)
    """
    #      (original - target)
    # rows:    (240 - 66) / 2 == 87
    # columns: (320 - 200) /2 == 60 
    return image[87:-87, 60:-60] 

# You might want to grab more of the image area and shrink it 
# down (instead of just cropping the center of the image out), e.g.,
# cv.resize(image,(0,0), fx=0.4, fy=0.4, interpolation=cv.INTER_AREA )
def shrink(image):
    return cv.resize(image, (200,66), cv.INTER_AREA)

# you can try using a larger crop:
def pilotnet_crop_large(image):
    """
    assumes 320x240 input, resizes to 280x120
    """
    #       (original - target)
    # rows:      (240 - 120) / 2 == 60
    # columns:   (320 - 280) /2 == 35
    return image[60:-60, 20:-20] 

# this is the function that will be used to preprocess your images when training the 
# model below - if you want to use a different crop edit the crop helper that's called here!
def preprocess(image, use_full_frame=False):
    if use_full_frame:
        return shrink(image)
    return crop(image)

In [5]:
# TODO: fill in these functions, which should take in an OpenCV image and return a new image 
# that represents rotating the input image to the left and the right respectively. 
# You may find the OpenCV functions getPerspectiveTransform and warpPerspective helpful!

def rotate_car_left(img):
    raise NotImplementedError

def rotate_car_right(img):
    raise NotImplementedError

In [ ]:
# load un-augmented training data

# add paths to all your training data csv files to this list:
driving_data = ["/content/drive/MyDrive/training_data/take1.csv"]

imgs   = []
ngls   = []
speeds = []

for f in driving_data:
    parent_dir = os.path.dirname(f) + os.path.sep
    with open(f) as fh:
        for line in fh:
            l = line.split(',')

            speed = l[2]
            speed = float(speed)

            speeds.append(speed)

            img = l[0].split('/')[-1]
            imgs.append(parent_dir + img)

            ngl = l[1]
            ngl = float(ngl)
            ngls.append(ngl)

print("total data", len(imgs))

# visualize the results of your fake left and right camera rotations on a training data image, 
# as well as the cropped versions:
print("right cam:")
cv2_imshow(rotate_car_left(cv.imread(imgs[0])))
cv2_imshow(preprocess(rotate_car_left(cv.imread(imgs[0]))))
print("left cam:")
cv2_imshow(rotate_car_right(cv.imread(imgs[0])))
cv2_imshow(preprocess(rotate_car_right(cv.imread(imgs[0]))))
print("center cam:")
cv2_imshow(cv.imread(imgs[0]))
cv2_imshow(preprocess(cv.imread(imgs[0])))

In [ ]:
# example of histograms that can help you examine what sort of steering angles and speeds your training data contains
# (note that unless you're trying to modify the model architecture to also predict speed, your speed histogram should only show one value)

fig, ax = plt.subplots(ncols=2)
ax[0].hist(ngls, bins=50);
ax[1].hist(speeds, bins=50);
ax[0].set_title("steering angles")
ax[1].set_title("speeds")
fig.set_size_inches(8,5);

In [ ]:
# example of visualizing a random preprocessed image from your training data

random_id = np.random.randint(0, len(imgs) - 1)

img = preprocess(cv.imread(imgs[random_id], cv.IMREAD_COLOR))

print("angle: {:02.3f}".format(ngls[random_id]))
print("image size:", img.shape)

rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
fig, ax = plt.subplots()
ax.imshow(rgb)
fig.set_size_inches(15,5);

In [ ]:
'''
IMAGE AUGMENTATIONS

Here are some examples of image augmentation. One simply increases the brightness ("value" channel in the HSV colorspace), 
another perturbs the gamma value of the input image. Try other augmentation techniques to increase the model's robustness. 

Note that it can take some experimentation to determine which augmentations help - some augmentation strategies might 
hurt performance. For instance, would it make sense to flip images horizontally or vertically?
'''

def increase_brightness(img):
    # perceptually a bit more uniform than perturb_gamma
    value = np.random.randint(20,60)
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv.merge((h, s, v))
    img = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return img

def make_random_gamma():
    random_gamma = np.random.normal(1, 0.3)
    random_gamma = np.clip(random_gamma, 0.5, 2)
    return random_gamma

def perturb_gamma(img): # https://stackoverflow.com/a/51174313
    gamma = make_random_gamma()
    invGamma = 1.0 / gamma
    table = np.array([
        ((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)])
    return cv.LUT(img, table.astype(np.uint8))

In [ ]:
# augment the training data - can take a while, so only rerun this cell if you don't have augmented data already saved in a csv

new_imgs = []
angles = []

for i in range(len(imgs)):
    print("augmenting img {} out of {}".format(i + 1, len(imgs)))

    img = imgs[i]
    angle = ngls[i]

    new_imgs.append(img)
    angles.append(angle)

    bgr = cv.imread(img, cv.IMREAD_COLOR)

    bright = increase_brightness(bgr)
    name = img.replace(".jpg", "_bright.jpg")
    cv.imwrite(name, bright)
    new_imgs.append(name)
    angles.append(angle)

    gamma = perturb_gamma(bgr)
    name = img.replace(".jpg", "_gamma.jpg")
    cv.imwrite(name, gamma)
    new_imgs.append(name)
    angles.append(angle)

    rotate_left = rotate_car_left(bgr)
    name = img.replace(".jpg", "_rightcam.jpg")
    cv.imwrite(name, rotate_left)
    new_imgs.append(name)
    angles.append(angle - OFFSET_STEERING_ANGLE)

    rotate_right = rotate_car_right(bgr)
    name = img.replace(".jpg", "_leftcam.jpg")
    cv.imwrite(name, rotate_right)
    new_imgs.append(name)
    angles.append(angle + OFFSET_STEERING_ANGLE)

    # TODO: consider also augmenting left and right images with increased brightness, gamma perturbation, etc
    # (right now only the center image is given these augmentations)

imgs = new_imgs
ngls = angles

In [ ]:
# save augmented data for reloading later - make sure you delete the old one first if you're trying to save new augmented data
with open("/content/drive/MyDrive/training_data/augmented_data.csv", "a") as f:
    for i in range(len(imgs)):
        f.write("{},{}\n".format(imgs[i], ngls[i]))

In [ ]:
# reload augmented data from csv - fill in correct filename below

imgs = []
ngls = []

f = "/content/drive/MyDrive/training_data/augmented_data.csv"
parent_dir = os.path.dirname(f) + os.path.sep
with open(f) as fh:
    for line in fh:
        l = line.split(',')

        img = l[0].split('/')[-1]
        imgs.append(parent_dir + img)

        ngl = l[1]
        ngl = float(ngl)
        ngls.append(ngl)

print(len(imgs), len(ngls))

In [ ]:
# split data into training and validation subsets
# optionally randomize order and/or use a fixed seed
# you can also play with the fraction of the data reserved for validation

VAL_SIZE_FRACTION = 0.10
SEED = 56709

X_train, X_valid, y_train, y_valid = train_test_split(
    imgs, 
    ngls, 
    test_size=VAL_SIZE_FRACTION, 
    shuffle=True #False - TODO
)
#,random_state=SEED)

print(len(X_train), len(X_valid))

In [ ]:
# feel free to adjust the dropout rate, as well as play with the architecture of the model

def build_model(dropout_rate=0.5):
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE)) # normalize the data
    model.add(Conv2D(24, (5,5), strides=(2, 2), activation='elu'))
    model.add(Conv2D(36, (5,5), strides=(2, 2), activation='elu'))
    model.add(Conv2D(48, (5,5), strides=(2, 2), activation='elu'))
    model.add(Conv2D(64, (3,3), activation='elu'))
    model.add(Conv2D(64, (3,3), activation='elu'))
    model.add(Dropout(dropout_rate)) 
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary() # prints out the model description
    return model

model = build_model()

In [ ]:
# explore different learning rates and feel free to adjust the loss and optimizer as well!

model.compile(loss='mean_squared_error', optimizer=Adam(lr=1.0e-4))

In [ ]:
# right now training and validation batches are treated equally - feel free to use the is_training argument to 
# implement different behavior for training and validation

def batch_generator(image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image given image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(len(image_paths)):
            
            image = cv.imread(image_paths[index])

            image = preprocess(image)

            groundtruth_steering_angle = steering_angles[index]
            
            images[i] = image
            steers[i] = groundtruth_steering_angle
            
            i += 1
            if i == batch_size:
                break
        yield images, steers

In [ ]:
# these histograms of steering angles and speeds can be useful for sanity checking and considering what you should set `OFFSET_STEERING_ANGLE` to

fig, ax = plt.subplots(ncols=2)
ax[0].hist(y_train, bins=50);
ax[0].set_title("training steering angles")
ax[1].hist(y_valid, bins=50);
ax[1].set_title("validation steering angles")
fig.set_size_inches(8,5);

Note that comparing training and validation loss is not a perfect measurement of how your model will perform. Solely using mean squared error on steering angles is perhaps a crude accuracy metric on this task. Also note that if your validation data set is not characteristic of your training data set (see histograms), then the training and validation losses may be tough to compare.

In [ ]:
# explore different batch sizes, steps per epoch, and epochs

BATCH_SIZE=20
model.fit_generator(generator=batch_generator(X_train, y_train, batch_size=BATCH_SIZE, is_training=True),
                    steps_per_epoch=2000,
                    epochs=2,
                    validation_data=batch_generator(X_valid, y_valid, batch_size=BATCH_SIZE, is_training=False),
                    validation_steps=len(X_valid) // BATCH_SIZE,
                    verbose=1);

In [ ]:
# save model for future reloading - change the name of the directory you'd like to save to
model.save("/content/drive/MyDrive/training_data/wall_follower_model")